# Trainingspipeline 2

- ohne Bigramme
- vector_size: 300
- window: 5
- seed: 1

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [70]:
print(stopwords)

['ad', 'al', 'allo', 'ai', 'agli', 'all', 'agl', 'alla', 'alle', 'con', 'col', 'coi', 'da', 'dal', 'dallo', 'dai', 'dagli', 'dall', 'dagl', 'dalla', 'dalle', 'di', 'del', 'dello', 'dei', 'degli', 'dell', 'degl', 'della', 'delle', 'in', 'nel', 'nello', 'nei', 'negli', 'nell', 'negl', 'nella', 'nelle', 'su', 'sul', 'sullo', 'sui', 'sugli', 'sull', 'sugl', 'sulla', 'sulle', 'per', 'tra', 'contro', 'io', 'tu', 'lui', 'lei', 'noi', 'voi', 'loro', 'mio', 'mia', 'miei', 'mie', 'tuo', 'tua', 'tuoi', 'tue', 'suo', 'sua', 'suoi', 'sue', 'nostro', 'nostra', 'nostri', 'nostre', 'vostro', 'vostra', 'vostri', 'vostre', 'mi', 'ti', 'ci', 'vi', 'lo', 'la', 'li', 'le', 'gli', 'ne', 'il', 'un', 'uno', 'una', 'ma', 'ed', 'se', 'perché', 'anche', 'come', 'dov', 'dove', 'che', 'chi', 'cui', 'non', 'più', 'quale', 'quanto', 'quanti', 'quanta', 'quante', 'quello', 'quelli', 'quella', 'quelle', 'questo', 'questi', 'questa', 'queste', 'si', 'tutto', 'tutti', 'a', 'c', 'e', 'i', 'l', 'o', 'ho', 'hai', 'ha', 'ab

In [3]:
df = pd.read_csv('../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [4]:
df.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
0,Espositivi.IV.4.Testo.txt,MIDIA,Ludovico Antonio Muratori,Antichità italiane,1700.0,1700-1750,espositivo,"﻿IV. 4. Ludovico Antonio Muratori, Antichità i...",8990.0,"﻿iv . 4 . Ludovico Antonio muratori , antichit..."
1,Poesia.IV.11.Testo.txt,MIDIA,Giambattista Felice Zappi,Poesie,1700.0,1700-1750,poesia,IV. 11. Rime degli arcadi: Tirsi Leucasio (Gio...,6113.0,iv . 11 . rima del arcade : tirsi leucasio ( g...
2,Personali.IV.15.Testo.txt,MIDIA,Pietro Giannone,Vita scritta da lui medesimo,1700.0,1700-1750,personale,[Proemio]\nPrendo a scrivere la mia vita e qua...,10118.0,[ proemio ] \n prendere a scrivere il mio vita...
3,Personali.IV.4.Testo.txt,MIDIA,Vincenzo da Filicaia,Lettere inedite a Lorenzo Magalotti,1700.0,1700-1750,personale,"IV. 4. Vincenzo da Filicaia, Lettere inedite a...",10073.0,"iv . 4 . Vincenzo da filicaia , lettere inedit..."
4,Poesia.IV.4.Testo.txt,MIDIA,Faustina Maratti Zappi,Poesie,1700.0,1700-1750,poesia,IV. 4. Rime degli Arcadi: Aglauro Cidonia (Fau...,3184.0,iv . 4 . rima del arcadi : aglauro cidonia ( f...


In [5]:
df.shape

(239270, 10)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1975']
df10 = df_periods['1976-2000']
df11 = df_periods['2001-2021']

In [9]:
df9.head()

,doc,source,author,title,year,period,text_type,text,words,lemmatized_text
75014,GBfare_randomsample.csv,Gutenberg,Arnaldo Bonaventura,"I Bagni di Lucca, Coreglia e Barga",1951.0,1951-1975,prosa letteraria,"Dopo varie vicende, alla morte di Paolo Guinig...",66.0,"dopo varie vicenda , alla morte di Paolo guini..."
75015,GBdi_randomsample.csv,Gutenberg,Arnaldo Bonaventura,"I Bagni di Lucca, Coreglia e Barga",1951.0,1951-1975,prosa letteraria,"Vi stava, in altri tempi, un vero e proprio er...",66.0,"vi stare , in altro tempo , uno vero e proprio..."
75016,GBanche_randomsample.csv,Gutenberg,Arnaldo Bonaventura,"I Bagni di Lucca, Coreglia e Barga",1951.0,1951-1975,prosa letteraria,"Porta Macchiaia, così chiamata perchè metteva ...",75.0,"porta macchiaia , così chiamare perchè mettere..."
75017,GBdi_randomsample.csv,Gutenberg,Arnaldo Bonaventura,"I Bagni di Lucca, Coreglia e Barga",1951.0,1951-1975,prosa letteraria,[Illustrazione: BAGNI DI LUCCA – STRETTE DI LI...,7.0,[ illustrazione : bagni di lucca – strette di ...
75018,GBanche_randomsample.csv,Gutenberg,Arnaldo Bonaventura,"I Bagni di Lucca, Coreglia e Barga",1951.0,1951-1975,prosa letteraria,Anche nei pressi di Limano le acque del fiume...,19.0,anche nel pressi di limano il acqua del fium...


## Training von Word2Vec

In [10]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [11]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 5                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 1                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

In [12]:
# Ordner anlegen zum Abspeichern von trainierten Modellen

if not os.path.exists('../trained_models'):
    os.makedirs('../trained_models')

### Zeitraum 1: 1700-1750

In [13]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [14]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 19 s


In [15]:
print(sentences1[3])

['oggetto', 'ammirazione', 'essere', 'antico', 'tempo', 'roma', 'roma', 'stendere', 'imperio', 'già', 'sopra', 'terra', 'alcun', 'scrittore', 'adulatoriamente', 'scrivere', 'volta', 'sì', 'bene', 'sopra', 'gran', 'parte', 'tre', 'parto', 'allora', 'conoscere', 'terra']


In [16]:
len(sentences1)

115393

In [17]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 57s


In [18]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('sconfiggere', 0.7794778943061829),
 ('peste', 0.7690707445144653),
 ('rumore', 0.760107159614563),
 ('spavento', 0.7593967318534851),
 ('riempire', 0.7498262524604797),
 ('strage', 0.7495648264884949),
 ('tumultuare', 0.748607873916626),
 ('macello', 0.7456192970275879),
 ('goti', 0.7441697716712952),
 ('trucidare', 0.7435092329978943)]

In [19]:
# trainiertes Modell speichern

w2v1.save(os.path.join('../trained_models/Word2Vec2', '2w2v1.model'))

### Zeitraum 2: 1751-1800

In [20]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [21]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 20.3 s


In [22]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 41s


In [23]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('tumulto', 0.8642402291297913),
 ('respingere', 0.8434235453605652),
 ('terror', 0.8331255316734314),
 ('cadde', 0.8318428993225098),
 ('sospingere', 0.8315478563308716),
 ('spavento', 0.8311995267868042),
 ('oppresso', 0.8298661708831787),
 ('infernale', 0.8281194567680359),
 ('sgherro', 0.8237597346305847),
 ('velare', 0.8227956295013428)]

In [24]:
w2v2.save(os.path.join('../trained_models/Word2Vec2', '2w2v2.model'))

### Zeitraum 3: 1801-1825

In [25]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [26]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 16.6 s


In [27]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 46s


In [28]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('forsennato', 0.8650603890419006),
 ('orrore', 0.8465152978897095),
 ('avversa', 0.8429762721061707),
 ('avvampare', 0.8328049778938293),
 ('turpe', 0.8311552405357361),
 ('languente', 0.8296940326690674),
 ('fantasma', 0.8290467262268066),
 ('furore', 0.8251278400421143),
 ('sovrumano', 0.8245025873184204),
 ('rissa', 0.8228349089622498)]

In [29]:
w2v3.save(os.path.join('../trained_models/Word2Vec2', '2w2v3.model'))

### Zeitraum 4: 1826-1850

In [30]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [31]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 15.4 s


In [32]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 40s


In [33]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('provocare', 0.8031339645385742),
 ('furore', 0.7924851179122925),
 ('rabbia', 0.790874719619751),
 ('gelosia', 0.7804804444313049),
 ('opprimere', 0.7772582769393921),
 ('fremito', 0.7754759788513184),
 ('frenare', 0.773889422416687),
 ('represso', 0.7591515779495239),
 ('crudeltà', 0.7585156559944153),
 ('rancore', 0.7579689621925354)]

In [34]:
w2v4.save(os.path.join('../trained_models/Word2Vec2', '2w2v4.model'))

### Zeitraum 5: 1851-1875

In [35]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [36]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.4 s


In [37]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 43s


In [38]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('furore', 0.839209258556366),
 ('raccapriccio', 0.8336207270622253),
 ('spavento', 0.8321473598480225),
 ('rimorso', 0.8232161998748779),
 ('atroce', 0.8229883313179016),
 ('straziare', 0.814719557762146),
 ('sgomento', 0.8120414614677429),
 ('tremendo', 0.8116239309310913),
 ('ira', 0.8087446689605713),
 ('disperazione', 0.8043317794799805)]

In [39]:
w2v5.save(os.path.join('../trained_models/Word2Vec2', '2w2v5.model'))

### Zeitraum 6: 1876-1900

In [40]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [41]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 18.5 s


In [42]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 49s


In [43]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('sgomento', 0.8218713998794556),
 ('furore', 0.8101630806922913),
 ('orrore', 0.7739326357841492),
 ('angoscioso', 0.7732362747192383),
 ('disperato', 0.7732051610946655),
 ('angoscia', 0.7697848677635193),
 ('invadere', 0.7610820531845093),
 ('invasare', 0.7582061290740967),
 ('spavento', 0.7578181028366089),
 ('crollo', 0.7551249265670776)]

In [44]:
w2v6.save(os.path.join('../trained_models/Word2Vec2', '2w2v6.model'))

### Zeitraum 7: 1901-1925

In [45]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [46]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 22.6 s


In [47]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 43s


In [48]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('sdegno', 0.8026636838912964),
 ('spavento', 0.80063396692276),
 ('subitaneo', 0.7930898666381836),
 ('indignazione', 0.7870234251022339),
 ('indicibile', 0.7829351425170898),
 ('allegrezza', 0.774941623210907),
 ('cordoglio', 0.7743913531303406),
 ('angoscioso', 0.7737253904342651),
 ('furore', 0.7707707285881042),
 ('brivido', 0.7665970325469971)]

In [49]:
w2v7.save(os.path.join('../trained_models/Word2vec2', '2w2v7.model'))

### Zeitraum 8: 1926-1950

In [50]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [51]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 19.5 s


In [52]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 55s


In [53]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.843616783618927),
 ('raccapriccio', 0.7936955094337463),
 ('orrore', 0.793317437171936),
 ('subitaneo', 0.7918689846992493),
 ('ribrezzo', 0.7847706079483032),
 ('spasimo', 0.7798827886581421),
 ('fremere', 0.7795377373695374),
 ('sgomento', 0.7781808376312256),
 ('indicibile', 0.7750077843666077),
 ('tremore', 0.7689642310142517)]

In [54]:
w2v8.save(os.path.join('../trained_models/Word2Vec2', '2w2v8.model'))

### Zeitraum 9: 1951-1975

In [55]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [56]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 19.6 s


In [57]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 50s


In [58]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.8054923415184021),
 ('immane', 0.7994385957717896),
 ('tremendo', 0.7935186624526978),
 ('smarrito', 0.7899277806282043),
 ('impotente', 0.7865541577339172),
 ('smarrimento', 0.7864586710929871),
 ('admeto', 0.7853696346282959),
 ('scherno', 0.7844949960708618),
 ('preda', 0.7832138538360596),
 ('sconforto', 0.7826411724090576)]

In [59]:
w2v9.save(os.path.join('../trained_models/Word2vec2', '2w2v9.model'))

### Zeitraum 10: 1976-2000

In [60]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [61]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 18.3 s


In [62]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 38s


In [63]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('incubo', 0.5719290375709534),
 ('repressione', 0.5627341866493225),
 ('massacro', 0.5614312887191772),
 ('persecuzione', 0.5486811995506287),
 ('serbe', 0.5484895706176758),
 ('intimidazione', 0.5472903251647949),
 ('rappresaglia', 0.5470618009567261),
 ('epidemia', 0.5420811772346497),
 ('sanguinario', 0.5386227369308472),
 ('bolscevico', 0.5381230711936951)]

In [64]:
w2v10.save(os.path.join('../trained_models/Word2Vec2', '2w2v10.model'))

### Zeitraum 11: 2001-2021

In [65]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [66]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 12.8 s


In [67]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 1min 56s


In [68]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('distruzione', 0.8287016749382019),
 ('barbaro', 0.8228227496147156),
 ('orrore', 0.8184574246406555),
 ('disperazione', 0.8181180357933044),
 ('seminare', 0.8147108554840088),
 ('odio', 0.8101599812507629),
 ('devastante', 0.7944847345352173),
 ('tormento', 0.7927058339118958),
 ('rtpre', 0.7882996797561646),
 ('jihad', 0.787747323513031)]

In [69]:
w2v11.save(os.path.join('../trained_models/Word2Vec2', '2w2v11.model'))